<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=a4c8c4bfeb1c268e2cbdd29020c3c3e6fdbd049e5d0572ff22b9584a5e4c6478
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [25]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
!wget -P -Q https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv -O forestfireinput.csv

--2022-11-08 21:51:43--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25425 (25K) [text/plain]
Saving to: ‘forestfireinput.csv’

estfireinput.csv    100%[===================>]  24.83K  --.-KB/s    in 0.002s  

2022-11-08 21:51:44 (15.6 MB/s) - ‘forestfireinput.csv’ saved [25425/25425]



In [4]:
schema = 'x_axis long, y_axis long, month string, day string, FFMC double, DMC double, DC double, temp double, RH double, wind double, rain double, area double'
forestfireinput = spark.read.csv('/content/forestfireinput.csv', schema=schema)

In [6]:
forestfireinput.select(forestfireinput.temp).show(4)

+----+
|temp|
+----+
| 5.1|
| 6.7|
| 6.7|
| 9.0|
+----+
only showing top 4 rows



In [8]:
from pyspark.sql.functions import upper
forestfireinput.withColumn('upper_month', upper(forestfireinput.month)).show(4)

+------+------+-----+---+----+----+-----+----+----+----+----+----+-----------+
|x_axis|y_axis|month|day|FFMC| DMC|   DC|temp|  RH|wind|rain|area|upper_month|
+------+------+-----+---+----+----+-----+----+----+----+----+----+-----------+
|     7|     5|  mar|fri|86.2|26.2| 94.3| 5.1| 8.2|51.0| 6.7| 0.0|        MAR|
|     7|     4|  oct|tue|90.6|35.4|669.1| 6.7|18.0|33.0| 0.9| 0.0|        OCT|
|     7|     4|  oct|sat|90.6|43.7|686.9| 6.7|14.6|33.0| 1.3| 0.0|        OCT|
|     8|     6|  mar|fri|91.7|33.3| 77.5| 9.0| 8.3|97.0| 4.0| 0.2|        MAR|
+------+------+-----+---+----+----+-----+----+----+----+----+----+-----------+
only showing top 4 rows



In [15]:
forestfireinput.select("temp").show(4)

+----+
|temp|
+----+
| 5.1|
| 6.7|
| 6.7|
| 9.0|
+----+
only showing top 4 rows



In [28]:
from pyspark.sql.functions import when 
forestfireinput.select("temp", when(forestfireinput.area > 0.0, "Burned").otherwise("Did not burn").alias("Did it burn?")).limit(10).toPandas()

,temp,Did it burn?
0,5.1,Did not burn
1,6.7,Did not burn
2,6.7,Did not burn
3,9.0,Burned
4,9.6,Did not burn
5,14.7,Did not burn
6,8.5,Did not burn
7,10.7,Did not burn
8,7.0,Did not burn
9,7.1,Did not burn


In [29]:
forestfireinput[forestfireinput.month.isin("jan", "feb", "mar")].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0
1,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2
2,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0
3,5,5,mar,sat,91.7,35.8,80.8,7.8,15.1,27.0,5.4,0.0
4,6,4,mar,wed,89.2,27.9,70.8,6.3,15.9,35.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,6,4,feb,tue,75.1,4.4,16.2,1.9,5.1,77.0,5.4,0.0
72,2,2,feb,sat,79.5,3.6,15.3,1.8,4.6,59.0,0.9,0.0
73,6,5,mar,mon,87.2,15.1,36.9,7.1,10.2,45.0,5.8,0.0
74,3,4,mar,wed,90.2,18.5,41.1,7.3,11.2,41.0,5.4,0.0


In [31]:
forestfireinput[forestfireinput.month.like("%r%")].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0
1,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2
2,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0
3,5,5,mar,sat,91.7,35.8,80.8,7.8,15.1,27.0,5.4,0.0
4,6,4,mar,wed,89.2,27.9,70.8,6.3,15.9,35.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58,6,5,mar,mon,87.2,15.1,36.9,7.1,10.2,45.0,5.8,0.0
59,3,4,mar,wed,90.2,18.5,41.1,7.3,11.2,41.0,5.4,0.0
60,6,5,mar,thu,91.3,20.6,43.5,8.5,13.3,27.0,3.6,0.0
61,6,3,apr,sun,91.0,14.6,25.6,12.3,13.7,33.0,9.4,0.0


In [34]:
forestfireinput[forestfireinput.month.like("o%")][forestfireinput.month.like('%t')].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0
1,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33.0,1.3,0.0
2,8,5,oct,mon,84.9,32.8,664.2,3.0,16.7,47.0,4.9,0.0
3,6,3,oct,tue,90.6,35.4,669.1,6.7,21.7,24.0,4.5,0.0
4,7,4,oct,fri,90.0,41.5,682.6,8.7,11.3,60.0,5.4,0.0
5,7,3,oct,sat,90.6,43.7,686.9,6.7,17.8,27.0,4.0,0.0
6,4,3,oct,sun,92.6,46.5,691.8,8.8,13.8,50.0,2.7,0.0
7,3,4,oct,sun,92.6,46.5,691.8,8.8,20.6,24.0,5.4,0.0
8,3,5,oct,wed,91.4,37.9,673.8,5.2,15.9,46.0,3.6,0.0
9,2,5,oct,sun,92.6,46.5,691.8,8.8,15.4,35.0,0.9,0.0


In [48]:
forestfireinput.filter(forestfireinput.month == 'oct').show(1)

+------+------+-----+---+----+----+-----+----+----+----+----+----+
|x_axis|y_axis|month|day|FFMC| DMC|   DC|temp|  RH|wind|rain|area|
+------+------+-----+---+----+----+-----+----+----+----+----+----+
|     7|     4|  oct|tue|90.6|35.4|669.1| 6.7|18.0|33.0| 0.9| 0.0|
+------+------+-----+---+----+----+-----+----+----+----+----+----+
only showing top 1 row



In [50]:
forestfireinput.groupBy("month", "day").avg("temp").show(10)

+-----+---+------------------+
|month|day|         avg(temp)|
+-----+---+------------------+
|  jun|mon| 6.566666666666666|
|  oct|sun|               8.8|
|  sep|fri|10.813157894736841|
|  jan|sun|               0.0|
|  oct|wed|               5.2|
|  apr|sun| 9.366666666666667|
|  feb|wed|               3.2|
|  mar|fri| 8.363636363636363|
|  sep|thu|  9.52857142857143|
|  jul|thu| 9.633333333333333|
+-----+---+------------------+
only showing top 10 rows



In [54]:
from pyspark.sql.functions import avg
forestfireinput.groupBy("month", "day").agg(avg("temp").alias("temperature")).orderBy("temperature", asc=False).show(10)

+-----+---+------------------+
|month|day|       temperature|
+-----+---+------------------+
|  jan|sun|               0.0|
|  nov|tue|               1.1|
|  feb|tue|               1.9|
|  dec|thu|               2.0|
|  feb|sat|2.0250000000000004|
|  apr|fri|               2.3|
|  dec|mon|               2.6|
|  dec|tue|               2.6|
|  apr|thu|               2.7|
|  feb|mon|2.7999999999999994|
+-----+---+------------------+
only showing top 10 rows



In [58]:
forestfireinput.groupBy("month").count().orderBy("count", asc=True).toPandas()

,month,count
0,nov,1
1,may,2
2,jan,2
3,apr,9
4,dec,9
5,oct,15
6,jun,17
7,feb,20
8,jul,32
9,mar,54
